<a href="https://colab.research.google.com/github/noportman/noportman/blob/main/USGS_station_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This code is created to determine the number of USGS gage stations are located in the State of Michigan.

Created: 7/12/2023
Modified: 7/12/2023
Author: Noah Portman

In [ ]:
# install dataretrieval packages

!pip install dataretrieval

In [ ]:
# install geemap packages

!pip install geemap

In [5]:
# import libraries
from dataretrieval import nwis
import geemap
import geopandas as gpd
import ee

# create a datafreame using the nwis package
df, md = nwis.what_sites(stateCd='MI', startDt='2011-01-01', EndDt='2023-07-11', siteType="LK")

# display the datafram
df

,agency_cd,site_no,station_nm,site_tp_cd,dec_lat_va,dec_long_va,coord_acy_cd,dec_coord_datum_cd,alt_va,alt_acy_va,alt_datum_cd,huc_cd
0,USGS,04035995,"LAKE GOGEBIC NEAR BERGLAND, MI",LK,46.588558,-89.547931,M,NAD83,1292.67,0.01,NAVD88,4020102.0
1,USGS,04037400,"CISCO LAKE NEAR WATERSMEET, MI",LK,46.252726,-89.452091,T,NAD83,1679.49,0.01,NAVD88,4020102.0
2,USGS,04057811,"GREENWOOD RESERVOIR NEAR GREENWOOD, MI",LK,46.442160,-87.800691,S,NAD83,1400.14,0.01,NAVD88,4030110.0
3,USGS,04058190,"SCHWEITZER RESERVOIR NEAR PALMER, MI",LK,46.416605,-87.646803,S,NAD83,1300.16,0.01,NAVD88,4030110.0
4,USGS,04097188,"AUSTIN LAKE NEAR KALAMAZOO, MI",LK,42.190877,-85.550834,S,NAD83,849.70,0.01,NAVD88,4050001.0
...,...,...,...,...,...,...,...,...,...,...,...,...
182,USGS,480557088391001,"SARGENT LAKE NR NE END OF LAKE AT ISLE ROYALE, MI",LK,48.099189,-88.652972,S,NAD83,701.00,4.30,NAVD88,4020300.0
183,USGS,480841088323401,"LAKE AHMIK NEAR WEST END AT ISLE ROYALE, MI",LK,48.146553,-88.543014,R,NAD83,622.00,4.30,NAVD88,4020300.0
184,USGS,480852088322901,"LAKE AHMIK NR CENTER OF LAKE ISLE ROYALE, MI",LK,48.147881,-88.541550,R,NAD83,622.00,4.30,NAVD88,4020300.0
185,USGS,480856088322001,"LAKE AHMIK NEAR SOUTH SHORE AT ISLE ROYALE, MI",LK,48.148975,-88.538933,R,NAD83,622.00,4.30,NAVD88,4020300.0


In [9]:
# Authenticate the Earth Engine API
ee.Authenticate()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=AuqLaqHG209wJzeLSPCmdyugBYwe8a1gwi7cKbuprgw&tc=ExN9cQ1euEPm6hlh0aSnzVRi9EKyHAT3ycozKqvuivk&cc=_c5MDFY2DHeJAYuc2AjZaCMGpuvIS25AKnBkTfQKerk

The authorization workflow will generate a code, which you should paste in the box below.
Enter verification code: 4/1AZEOvhW2fs483RFMCGauLkKAbFwxFIwPs9jmq1ydyNPm9LZL-Wy4MSWDBdQ

Successfully saved authorization token.


In [10]:
# Initialize the Earth Engine API
ee.Initialize()

In [6]:
# Drop rows with NaN values in 'dec_lat_va' and 'dec_long_va' columns
df_clean = df.dropna(subset=['dec_lat_va', 'dec_long_va'])

# Display the clean dataframe
df_clean

,agency_cd,site_no,station_nm,site_tp_cd,dec_lat_va,dec_long_va,coord_acy_cd,dec_coord_datum_cd,alt_va,alt_acy_va,alt_datum_cd,huc_cd
0,USGS,04035995,"LAKE GOGEBIC NEAR BERGLAND, MI",LK,46.588558,-89.547931,M,NAD83,1292.67,0.01,NAVD88,4020102.0
1,USGS,04037400,"CISCO LAKE NEAR WATERSMEET, MI",LK,46.252726,-89.452091,T,NAD83,1679.49,0.01,NAVD88,4020102.0
2,USGS,04057811,"GREENWOOD RESERVOIR NEAR GREENWOOD, MI",LK,46.442160,-87.800691,S,NAD83,1400.14,0.01,NAVD88,4030110.0
3,USGS,04058190,"SCHWEITZER RESERVOIR NEAR PALMER, MI",LK,46.416605,-87.646803,S,NAD83,1300.16,0.01,NAVD88,4030110.0
4,USGS,04097188,"AUSTIN LAKE NEAR KALAMAZOO, MI",LK,42.190877,-85.550834,S,NAD83,849.70,0.01,NAVD88,4050001.0
...,...,...,...,...,...,...,...,...,...,...,...,...
182,USGS,480557088391001,"SARGENT LAKE NR NE END OF LAKE AT ISLE ROYALE, MI",LK,48.099189,-88.652972,S,NAD83,701.00,4.30,NAVD88,4020300.0
183,USGS,480841088323401,"LAKE AHMIK NEAR WEST END AT ISLE ROYALE, MI",LK,48.146553,-88.543014,R,NAD83,622.00,4.30,NAVD88,4020300.0
184,USGS,480852088322901,"LAKE AHMIK NR CENTER OF LAKE ISLE ROYALE, MI",LK,48.147881,-88.541550,R,NAD83,622.00,4.30,NAVD88,4020300.0
185,USGS,480856088322001,"LAKE AHMIK NEAR SOUTH SHORE AT ISLE ROYALE, MI",LK,48.148975,-88.538933,R,NAD83,622.00,4.30,NAVD88,4020300.0


In [11]:
# Create a geopandas GeoDataFrame from the dataframe
gdf = gpd.GeoDataFrame(df_clean, geometry=gpd.points_from_xy(df_clean['dec_long_va'], df_clean['dec_lat_va']))

# Set the CRS for the GeoDataFrame
gdf.crs = 'EPSG:4326'  # Assuming WGS84 coordinates

# Convert the GeoDataFrame to a ee.FeatureCollection
fc = geemap.geopandas_to_ee(gdf)

# Create a geemap Map instance
Map = geemap.Map(center=[44.5, -85], zoom=6)

# Add the ee.FeatureCollection to the map
Map.addLayer(fc, {}, 'Lake Sites')

# Display the map
Map

Map(center=[44.5, -85], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(…